In [1]:
import os
import yaml
import csv
import math as m
import numpy as np
#import pandas as pd 

import re

#### READ ME ####

The purpose of this code is to take incomplete SLEAP data and complete it to make sure that there are no missed identities or parts and make sure that the position data is in such a form as to allow easy interpretation for feature extraction. I conceptualized this problem as working down the line of data with a small context window to make sure everything is in order. Procedurally this script should go down the whole list multiple times first finding missing identities, filling them, reordering things so that identities are consistant, merging identities, and then interpolating pointsso that we have two continuous paths in space. Also as of the 15th of January there are still a few things I need to do.

This is a different version of the script that takes the prediction on singles with a max identity set to 2.

In [2]:
## This is a search script that will find the files that match a pattern in the directory and return said matches

Rootdir= 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned' # replace this the scope of the search
OutputPath= os.getcwd() # can be replaced with a dedicated path ie 'C:/Users/Jume6823.AD.000/Downloads'

#SinglePattern= "L\d{1}\sTrap2\d{3}.analysis$"
#PairPattern = "L\d{1}\sTrap2\d{3}[+]\d{3}.analysis$"

def CSVSearch(sex:str, RootDir, Pair:bool):
    if Pair == 0:
        pattern= re.compile(r'L\d{1}\sTrap2\d{3}.analysis')
    else:
        pattern= re.compile(r'L\d{1}\sTrap2\d{3}[-+]\d{3}.analysis')
        print('hey')
    
    matching = []
    for path, directories, files in os.walk(RootDir):
        
        for file in files:
            if pattern.search(file):
                filepath= (path+os.sep+file).replace("\\","/")
                matching.append(filepath)
    return(matching)

def CSVSearch2(RootDir, Pair:bool):
    if Pair == 0:
        pattern= re.compile(r'L\d{1}\sTrap2\d{3}.analysis')
    else:
        pattern= re.compile(r'L\d{1}\sTrap2\d{3}[-+]\d{3}.analysis')
        print('hey')
    
    matching = []
    for path, directories, files in os.walk(RootDir):
        for file in files:
            filepath= (path+os.sep+file).replace("\\","/")
            matching.append(filepath)
    return(matching)

Singles = CSVSearch2(Rootdir,0)
print(len(Singles))

10


In [3]:
print(Singles)

['C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.000_20241028 Trap2social 5Vls M L1 Trap2168.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.001_20241101 Trap2social 5Vls M L5 Trap2155.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.002_20241113 Trap2social 5Vls F L1 Trap2158.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.003_20241115 Trap2social 5Vls F L3 Trap2158.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.004_20241115 Trap2social 5Vls F L3 Trap2161.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.005_20241118 Day3 327.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.006_20241120 Wt 5Vls Day5 C57338.analysis.csv', 'C:/Users/Public/SLEAP/SingleBehavior CSV/Uncleaned/SingleBehaviorSURF.007_20250228 Trap2 Social 5Vls Day5 237.analysis.cs

In [3]:
# This section of the code is for taking the csv files from the sleap labels and then just making it into a list I can work with 
numberofpoints= 6
pointlist = [[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]]
def FrameDistance(A,B): ### just taking the euclidiean distance of the distances between body points of the same type
    NoseD= m.dist([A[1][0],A[1][1]],[B[1][0],B[1][1]])
    EarLD= m.dist([A[2][0],A[2][1]],[B[2][0],B[2][1]])
    EarRD= m.dist([A[3][0],A[3][1]],[B[3][0],B[3][1]])
    HeadD =m.dist([A[4][0],A[4][1]],[B[4][0],B[4][1]])
    CenterD= m.dist([A[5][0],A[5][1]],[B[5][0],B[5][1]])
    TailD= m.dist([A[6][0],A[6][1]],[B[6][0],B[6][1]])
    TotalD= NoseD**2+EarLD**2+EarRD**2+HeadD**2+CenterD**2+TailD**2
    return(TotalD)


def SingleIdentityTracker(index):
    with open(Singles[index], 'r') as csvfile:
        pointreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        row_count= sum(1 for row in pointreader)
        
    ### Why these blocks have to be seperate I don't know but it works and I should probably switch to Pandas anyways but here we are
    
    with open(Singles[index], 'r') as csvfile:
        pointreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        #print(sum(1 for row in pointreader))
    
        ### The first two variables are the instance and frame respectively, from there the coordinates of the nose earL, earR, head, body center and tail base
        pointlist = [[[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0],[0,0,0]] for i in range(row_count)]
        j=0
        ### This is just copying over the first row
        FirstRow = next(pointreader)
        #print(FirstRow)
        #print(len(FirstRow))
        ### This is just making a new list that is more easy to work with
        for row in pointreader:
            i=0
            for col in row:
                if j !=-1:
                    ### The first index is for the track which is a string by default weirdly enough
                    if i == 0:
                        pointlist[j][0][0]= int(''.join([char for char in col[::-1] if char.isdigit()])[::-1]) ##this just grabs the last digits in the string
                    if i == 1:
                        pointlist[j][0][1]= int(col)
                    if i % 3 == 0 and i!=0:   
                        if col !='':
                            pointlist[j][int(i/3)][0]= float(col)
                    if i % 3 == 1 and i!=1:
                        if col !='':
                            pointlist[j][int((i-1)/3)][1]= float(col)
                    if i % 3 == 2:
                        if col !='':
                            pointlist[j][int((i-1)/3)][2]= float(col)
                i=i+1
            j=j+1
        #print(j)
        
        
        ### This checks for missing identities
        
        MissingId = []
        currentframe = 0
        for i in range(len(pointlist)-1):
            if pointlist[i][0][1] == currentframe:
                currentframe += 1
            else:
                for j in range(pointlist[i][0][1]-currentframe):
                    MissingId.append([i,currentframe, pointlist[i][0][1]])
                    currentframe += 1
                currentframe +=1
        NewPointList= pointlist
        
        #print(MissingId)
        #print(len(MissingId), "identities are missing ")
        for i in reversed(range(len(MissingId))):
            ### this just makes sure that the id we are putting back is the correct one and not the one picked up earlier 
            NewPointList.insert(MissingId[i][0],[[MissingId[i][2],MissingId[i][1],0.0],[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0],[0.0,0.0,0.0]])
        i=0
        
        #print(NewPointList[2])
        ### This is then linearly bridging any missing values by going along the array until a problem point is detected and then tracing back to the last entry and then foward until the next unempty entry and then interpolating the points linearly
        for i in range(len(NewPointList)):
            for j in range(6):
                ### This checks if there are any zero entries in the list
                if NewPointList[i][j+1][0] == 0.0 or NewPointList[i][j+1][1] == 0.0:
                    anchor = NewPointList[i-1][j+1]
                    #print("anchor is" + str(anchor))
                    scope = i
                    ### This looks forward for the next valid point going every row
                    while scope < (len(NewPointList)-1):
                        scope += 1
                        if NewPointList[scope][j+1][0] != 0.0:
                            endpoint = NewPointList[scope][j+1]
                            #print([j,i,scope,endpoint])
                            length = int(scope-i)
                            #print(length)
                            break 
                    ### This is for the weird case in which there is no endpoint and so then we  just go backwards and find the last valid point and copy it over. We should discuss with Dr. Molas if instead the data should be cropped in that case.
                    else:
                        #print("endpoint missing starting from " + str(i))
                        backscope = i
                        while backscope <= (len(NewPointList)):
                            backscope -= 1
                            if NewPointList[backscope][j+1][0] != 0.0:
                                endpoint = NewPointList[backscope][j+1]
                                ## This just tells the next function to take the rest of the missing values and fill them in with the last found value
                                length = int((len(NewPointList))-i)
                                break 
                    #print(anchor,endpoint)
                    ### This is simply a linear interpolation of points. Other ways of fittig could also be used here.
                    for k in range(length):
                        ratio= (k+1)/(length+1)
                        NewPointList[i+(k)][j+1][0]= anchor[0]*(1-ratio)+endpoint[0]*ratio
                        NewPointList[i+(k)][j+1][1]= anchor[1]*(1-ratio)+endpoint[1]*ratio
                        NewPointList[i+(k)][j+1][2]= anchor[2]*(1-ratio)+endpoint[2]*ratio
                    TempList = []
                    for k in range(length):
                        TempList.append(NewPointList[i+(k)][j+1][0])
                    
                    #print(i+length)
        pointlist=NewPointList
        NewNameIndex= Singles[index].rfind('_')
        #print(NewNameIndex)
        NewName= Singles[index][NewNameIndex+1:]
        print(NewName)
        #print(pointlist[0])
        data = [FirstRow]
        with open(OutputPath+os.sep+NewName, 'w', newline ='') as f:
            ### Not fully sure what dialect to use here(a dialect is essentially different protocols for interpreting csv) but excel should be fine I imagine.
            writer = csv.writer(f,dialect='excel')
            
            #for row in pointreader:
              #  new_row = row 
              #  writer.writerow(new_row)
            j=0
            ## Just skipping the first 0 line
            for i in range(len(pointlist)-1):
                data.append([0] * 21)
                data[i+1][0]= 'track_0' ## He we force it be 1 identity
                data[i+1][1]= pointlist[i][0][1]
                data[i+1][2]= pointlist[i][0][2]
                for j in range(6):
                    data[i+1][3+(3*j)] = pointlist[i][j+1][0]
                    data[i+1][4+(3*j)] = pointlist[i][j+1][1]
                    data[i+1][5+(3*j)] = pointlist[i][j+1][2]
            #print(data[0])
            #print(data[1])
            writer.writerows(data)
            #print(MissingId)
            #print(len(MissingId), "identities are missing ")


In [4]:
#### Run up to this block
for i in range(len(Singles)):
    SingleIdentityTracker(i)

20241028 Trap2social 5Vls M L1 Trap2168.analysis.csv
20241101 Trap2social 5Vls M L5 Trap2155.analysis.csv
20241113 Trap2social 5Vls F L1 Trap2158.analysis.csv
20241115 Trap2social 5Vls F L3 Trap2158.analysis.csv
20241115 Trap2social 5Vls F L3 Trap2161.analysis.csv
20241118 Day3 327.analysis.csv
20241120 Wt 5Vls Day5 C57338.analysis.csv
20250228 Trap2 Social 5Vls Day5 237.analysis.csv
20250319 Tam Control 5Vls Day3 Trap2 251.analysis.csv
20250320 Tam Control 5Vls Day4 Trap2 248.analysis.csv
